In [1]:
import psycopg2
from psycopg2  import extras
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, Date

DATABASE = 'postgresql'
USER = 'root'
PASSWORD = 'root'
HOST = 'postgres' # shared-networkに所属しているpostgres containerを指定できる
PORT = '5432'
DB_NAME = 'sandbox'

CONNECT_STR = '{}://{}:{}@{}:{}/{}'.format(DATABASE, USER, PASSWORD, HOST, PORT, DB_NAME)
engine = create_engine(CONNECT_STR)

conn = engine.connect()
res = conn.execute("select datname, datdba, encoding, datcollate, datctype from pg_database;")

for row in res:
    print(row)

('postgres', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('sandbox', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template1', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')
('template0', 10, 6, 'ja_JP.UTF-8', 'ja_JP.UTF-8')


In [2]:
# create table
tablename_a = 'x008_class_a'
a_q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename_a)
a_res = conn.execute(a_q1)

a_q2 = """
CREATE TABLE  {} (
    name varchar(256) NOT NULL,
    age  integer,
    city varchar(256)
);
""".format(tablename_a)

tablename_b = 'x008_class_b'
b_q1 = "SELECT * FROM information_schema.tables WHERE table_name = '{}';".format(tablename_b)
b_res = conn.execute(b_q1)

b_q2 = """
CREATE TABLE  {} (
    name varchar(256) NOT NULL,
    age  integer,
    city varchar(256)
);
""".format(tablename_b)

if sum(1 for _ in a_res) == 0:
    print("create")
    conn.execute(a_q2)
    res = conn.execute(a_q1)
    sum(1 for _ in res)

if sum(1 for _ in b_res) == 0:
    print("create")
    conn.execute(b_q2)
    res = conn.execute(b_q1)
    sum(1 for _ in res)

In [3]:
# insert data
data = (
    ('ブラウン', 22, '東京'),
    ('ラリー', 19, '埼玉'),
    ('ボギー', 21, '千葉'),
)

conn.execute("DELETE FROM {};".format(tablename_a))

metadata = MetaData()
tbl = Table(tablename_a, metadata,
            Column('name', String),
            Column('age', Integer),
            Column('city', String),
           )

for i in data:
    ins = tbl.insert().values(name=i[0], age=i[1], city=i[2])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename_a))
for i in res:
    print(i)

data = (
    ('斎藤', 22, '東京'),
    ('田尻', 23, '東京'),
    ('山田', None, '東京'),
    ('和泉', 18, '千葉'),
    ('武田', 20, '千葉'),
    ('石川', 19, '神奈川'),

)

conn.execute("DELETE FROM {};".format(tablename_b))

metadata = MetaData()
tbl = Table(tablename_b, metadata,
            Column('name', String),
            Column('age', Integer),
            Column('city', String),
           )

for i in data:
    ins = tbl.insert().values(name=i[0], age=i[1], city=i[2])
    conn.execute(ins)

res = conn.execute("SELECT * FROM {};".format(tablename_b))
for i in res:
    print(i)

('ブラウン', 22, '東京')
('ラリー', 19, '埼玉')
('ボギー', 21, '千葉')
('斎藤', 22, '東京')
('田尻', 23, '東京')
('山田', None, '東京')
('和泉', 18, '千葉')
('武田', 20, '千葉')
('石川', 19, '神奈川')


In [4]:
# NOT INとNOT EXISTは同値ではない

## NOT IN

q = f"""
select * from {tablename_a}
where age not in
    (select age
        from {tablename_b}
        where city = '東京'
    );
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


# 実際は以下のようになっている、where unknownとなる
q = f"""
select * from {tablename_a}
where age not in
    (22, 23, null
    );
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)

    
## NOT EXSIST

q = f"""
select * from {tablename_a} a
where not exists
    (select *
        from {tablename_b} b
        where a.age = b.age
        and b.city = '東京');
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select * from x008_class_a
where age not in
    (select age
        from x008_class_b
        where city = '東京'
    );


select * from x008_class_a
where age not in
    (22, 23, null
    );


select * from x008_class_a a
where not exists
    (select *
        from x008_class_b b
        where a.age = b.age
        and b.city = '東京');

('ラリー', 19, '埼玉')
('ボギー', 21, '千葉')


In [5]:
# coalesceとnullif

q = f"""
select *, nullif(coalesce(null, 'abc'), 'abc') as hoge
from {tablename_b};
"""

print(q)

res = conn.execute(q)
for i in res:
    print(i)


select *, nullif(coalesce(null, 'abc'), 'abc') as hoge
from x008_class_b;

('斎藤', 22, '東京', None)
('田尻', 23, '東京', None)
('山田', None, '東京', None)
('和泉', 18, '千葉', None)
('武田', 20, '千葉', None)
('石川', 19, '神奈川', None)
